In [2]:
import pickle
import numpy as np
import pandas as pd

# Profiles

In [3]:
with open('./save_feat/test_feature_r100_tiny_face_profiles.pickle', 'rb') as f:
    profiles_feats = pickle.load(f)

In [4]:
len(profiles_feats.keys())

4443

In [5]:
np.array(list(profiles_feats.values())).squeeze(1).shape

(4443, 512)

In [6]:
profiles_face_ids = []
profiles_image_names = []
for k in profiles_feats.keys():
    profiles_face_ids.append(k.split('_')[0])
    profiles_image_names.append(k)


In [7]:
profiles_df = pd.DataFrame({'id':profiles_face_ids, 'feat':list(profiles_feats.values()), 'image_name':profiles_image_names})

In [8]:
profiles_df

,id,feat,image_name
0,4470,"[[0.0032535982, 0.087296896, -0.0054867226, -0...",4470_3.jpg
1,1737,"[[0.023858232, -0.013248299, 0.029257584, 0.00...",1737_2.jpg
2,390,"[[0.049199242, 0.009060843, 0.059195463, 0.043...",390_11.jpg
3,4985,"[[-0.035605703, 0.02781184, 0.009004803, -0.02...",4985_1.jpg
4,1667,"[[-0.0028737723, 0.06858538, 0.049484707, -0.0...",1667_2.jpg
...,...,...,...
4438,961,"[[-0.02176254, -0.010053865, -0.04326786, -0.0...",961_6.jpg
4439,641,"[[0.018750526, 0.040325377, -0.020772193, -0.0...",641_11.jpg
4440,3041,"[[0.03290178, 0.014625239, 0.021345284, 0.1004...",3041_1.jpg
4441,4870,"[[0.04556825, -0.05682025, 0.030321093, 0.0209...",4870_2.jpg


In [9]:
profiles_mat = np.array(profiles_df.feat.to_list()).squeeze(1)

In [10]:
profiles_mat.shape

(4443, 512)

# Test Image

In [11]:
with open('./save_feat/test_feature_r100_tiny_face_test.pickle', 'rb') as f:
    test_feats = pickle.load(f)

In [12]:
len(test_feats.keys())

3728

In [13]:
np.array(list(test_feats.values())).squeeze(1).shape

(3728, 512)

In [14]:
test_face_ids = []
test_image_names = []
for k in test_feats.keys():
    test_face_ids.append(k.split('_')[0])
    test_image_names.append(k)

In [15]:
test_df = pd.DataFrame({'id':test_face_ids, 'feat':list(test_feats.values()), 'image_name':test_image_names})

In [16]:
test_df

,id,feat,image_name
0,4158,"[[-0.02561251, -0.022131706, 0.026354099, -0.0...",4158_6.jpg
1,3510,"[[-0.020593157, 0.013408898, 0.015119124, -0.0...",3510_2.jpg
2,5477,"[[-0.023336, -0.009472727, 0.018649304, -0.005...",5477_2.jpg
3,4020,"[[-0.0010503666, 0.029097129, 0.015072649, -0....",4020_3.jpg
4,2353,"[[-0.018030383, 0.11625832, 0.016371198, 0.031...",2353_4.jpg
...,...,...,...
3723,3190,"[[0.041545726, -0.017554404, 0.034322515, 0.04...",3190_1.jpg
3724,159,"[[0.07334582, -0.018404385, 0.09697984, 0.0787...",159_10.jpg
3725,3269,"[[0.007111652, 0.027439773, 0.026061954, 0.012...",3269_2.jpg
3726,5004,"[[0.053424664, 0.0048460322, -0.014051733, -0....",5004_1.jpg


In [17]:
test_mat = np.array(test_df.feat.to_list()).squeeze(1)

In [18]:
test_mat.shape

(3728, 512)

# Run Compute Sims

In [22]:
sims = (np.dot(test_mat, profiles_mat.T) + 1.0) / 2.0

In [23]:
sims.shape

(3728, 4443)

In [24]:
sims

array([[0.5280964 , 0.54059386, 0.48258016, ..., 0.550255  , 0.5349394 ,
        0.49423137],
       [0.60166776, 0.5827564 , 0.5096101 , ..., 0.5547889 , 0.597708  ,
        0.54188013],
       [0.5274681 , 0.50776964, 0.5425076 , ..., 0.5373103 , 0.5545709 ,
        0.54574186],
       ...,
       [0.53712183, 0.52558434, 0.6055504 , ..., 0.53892684, 0.54863894,
        0.55042243],
       [0.58168745, 0.62024814, 0.49527112, ..., 0.53765905, 0.46174678,
        0.5806039 ],
       [0.47943234, 0.54336166, 0.5684128 , ..., 0.6024626 , 0.48321474,
        0.5312079 ]], dtype=float32)

In [63]:
m_AP = 0
empty = 0
true_positive = 0
false_positive = 0
false_nagetive = 0

for i in range(sims.shape[0]):
    index = np.argmax(sims[i])
    if sims[i][index] < 0.73:
        false_nagetive += 1
    else:
        if test_df.iloc[i].id == profiles_df.iloc[index].id:
            true_positive += 1
        else:
            false_positive += 1

precision = true_positive / (true_positive + false_positive)
recall = true_positive / (true_positive + false_nagetive)
accuracy = true_positive / sims.shape[0]

In [64]:
precision

0.6145038167938931

In [65]:
recall

0.676729079098204

In [66]:
accuracy

0.4750536480686695